In [3]:
import praw
import json
import string


# function to preprocess text for Misra-Gries algorithm
def preprocess(text):
    # remove links
    text = ' '.join(word for word in text.split() if 'http' not in word)

    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove numbers
    text = text.translate(str.maketrans('', '', string.digits))
    # remove whitespace
    text = text.strip()
    # convert to lowercase
    text = text.lower()
    # remove ellipsis
    text = text.replace('...', '')
    # remove newlines
    text = text.replace('\n', '')
    # remove tabs
    text = text.replace('\t', '')
    # remove multiple spaces
    text = ' '.join(text.split())
    # remove leading and trailing whitespace
    text = text.strip()
    # remove apostrophes
    text = text.replace('’', '')
    
    return text



# load credentials from file
with open('client_secret.json') as f:
    data = json.load(f)
    client_id = data['client_id']
    client_secret = data['client_secret']
    user_agent = data['user_agent']

reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

subreddits = reddit.subreddits.popular(limit=25)
for subreddit in subreddits:
    subreddit_top_comments = []
    # Get the top 10 hot posts from the subreddit
    hot_posts = subreddit.hot(limit=1000)
    for post in hot_posts:
        # Get top 10 comments from the post
        comments = post.comments.list()[:100]
        for comment in comments:
            subreddit_top_comments.append(comment.body)

    # preprocess text
    subreddit_top_comments = [preprocess(comment) for comment in subreddit_top_comments]
    # write to file with subreddit name
    with open(subreddit.display_name + '.txt', 'w') as f:
        for comment in subreddit_top_comments:
            f.write(comment + '\n')




KeyboardInterrupt: 

In [15]:
subreddit = reddit.subreddit('all').hot(limit=1)
# list all attributes of the subreddit object
print(dir(subreddit))

# get top 25 subreddits
top_subreddits = reddit.subreddits.popular(limit=25)
for subreddit in top_subreddits:
    print(subreddit.url)


['__abstractmethods__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_exhausted', '_extract_sublist', '_is_protocol', '_list_index', '_listing', '_next_batch', '_reddit', '_safely_add_arguments', 'limit', 'params', 'parse', 'url', 'yielded']
/r/Home/
/r/AskReddit/
/r/funny/
/r/Damnthatsinteresting/
/r/worldnews/
/r/AmItheAsshole/
/r/interestingasfuck/
/r/antiwork/
/r/facepalm/
/r/gaming/
/r/Unexpected/
/r/pics/
/r/WhitePeopleTwitter/
/r/LivestreamFail/
/r/pcmasterrace/
/r/movies/
/r/therewasanattempt/
/r/leagueoflegends/
/r/diablo4/
/r/videos/
/r/DestinyTheGame/
/r/PublicFreakout/
/r/mi